# CBA / ANZ Balance Sheet

## Setup

In [1]:
# Import Libraries
import time
from datetime import datetime
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo

## Delete collection before inserting new data

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
client.Stocks_db.balance_sheet.drop()

## CBA Balance_sheet statement table

In [3]:
############# Extract #############
# Read the tables in the HTML page
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

bank='CBA'
url = f'https://au.finance.yahoo.com/quote/{bank}.AX/balance-sheet?p={bank}.AX'
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

# Get table headers
balance_sheet_headers = soup.find_all('div', class_='D(tbr) C($primaryColor)')
# Get table rows
balance_sheet_rows = soup.find_all('div', class_='D(tbr) fi-row Bgc($hoverBgColor):h')

# Quit the browser
browser.quit()

# Get the table headers
table_headers_list = []
for header in balance_sheet_headers:
    for span in header.find_all('span'):
        table_headers_list.append(span.text)

# Get all the table rows
table_rows = []
row_list = []
for rows in balance_sheet_rows:
    for fields in rows:
        row_list.append(fields.text)
    # Add row to table rows
    table_rows.append(row_list)
    # Reset the row list for the next row
    row_list = []
    
# Convert the result into a DataFrame
balance_table = pd.DataFrame(table_rows, columns=table_headers_list)




[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/ziliangzhang/.wdm/drivers/chromedriver/mac64/95.0.4638.17/chromedriver] found in cache


## Transform Data

In [4]:
############# Transform #############
# Remove all the rows without values based on any of the financial year columns
balance_table_new = balance_table[balance_table['29/06/2021']!=''].copy()
# Replace all fields containing '-' with 0
balance_table_new = balance_table_new.replace('-',0)
# Set the index to Breakdown
balance_table_new = balance_table_new.set_index('Breakdown')

# Divide into series by financial year
#2017-2018
balance_table_new_17to18 = balance_table_new[['29/06/2018']].copy()
#2018-2019
balance_table_new_18to19 = balance_table_new[['29/06/2019']].copy()
#2019-2020
balance_table_new_19to20 = balance_table_new[['29/06/2020']].copy()
#2020-2021
balance_table_new_20to21 = balance_table_new[['29/06/2021']].copy()


## Load data

In [6]:
############# Load #############

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

cba_dict = {}

cba_balance_reports = {'29/06/2018':balance_table_new_17to18.to_dict()['29/06/2018'],
                         '29/06/2019':balance_table_new_18to19.to_dict()['29/06/2019'],
                         '29/06/2020':balance_table_new_19to20.to_dict()['29/06/2020'],
                         '29/06/2021':balance_table_new_20to21.to_dict()['29/06/2021']
                      }


cba_dict["CBA"] = cba_balance_reports

client.stocks_db.balance_sheet.insert_one(cba_dict)


## ANZ Balance Sheet table

## Extrat data

In [7]:
# Read the tables in the HTML page
# Scrape the Data
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

bank='ANZ'
url = f'https://au.finance.yahoo.com/quote/{bank}.AX/balance-sheet?p={bank}.AX&.tsrc=fin-srch'
browser.visit(url)

time.sleep(1)

html = browser.html
soup = bs(html, "html.parser")

# Get table headers
balance_sheet_headers = soup.find_all('div', class_='D(tbr) C($primaryColor)')
# Get table rows
balance_sheet_rows = soup.find_all('div', class_='D(tbr) fi-row Bgc($hoverBgColor):h')

# Quit the browser
browser.quit()

# Get the table headers
table_headers_list = []
for header in balance_sheet_headers:
    for span in header.find_all('span'):
        table_headers_list.append(span.text)

# Get all the table rows
table_rows = []
row_list = []
for rows in balance_sheet_rows:
    for fields in rows:
        row_list.append(fields.text)
    # Add row to table rows
    table_rows.append(row_list)
    # Reset the row list for the next row
    row_list = []
    
# Convert the result into a DataFrame
balance_table = pd.DataFrame(table_rows, columns=table_headers_list)


[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [/Users/ziliangzhang/.wdm/drivers/chromedriver/mac64/95.0.4638.17/chromedriver] found in cache


In [10]:

# Remove all the rows without values based on any of the financial year columns
balance_table_new = balance_table[balance_table['29/09/2020']!=''].copy()
# Replace all fields containing '-' with 0
balance_table_new = balance_table_new.replace('-',0)
# Set the index to Breakdown
balance_table_new = balance_table_new.set_index('Breakdown')

# Divide into series by financial year
#2017-2018
balance_table_new_17to18 = balance_table_new[['29/09/2018']].copy()
#2018-2019
balance_table_new_18to19 = balance_table_new[['29/09/2019']].copy()
#2019-2020
balance_table_new_19to20 = balance_table_new[['29/09/2020']].copy()
#2020-2021
# balance_table_new_20to21 = balance_table_new[['29/06/2021']].copy()

In [11]:

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

anz_dict = {}
balance_sheet_dict = {}

anz_balance_reports = {'29/06/2018':balance_table_new_17to18.to_dict()['29/09/2018'],
                         '29/06/2019':balance_table_new_18to19.to_dict()['29/09/2019'],
                         '29/06/2020':balance_table_new_19to20.to_dict()['29/09/2020']
                      }


balance_sheet_dict["ANZ_Balance_Sheet"] = anz_balance_reports
anz_dict["ANZ"] = balance_sheet_dict["ANZ_balance_sheet"]

client.Stocks_db.balance_sheet.insert_one(anz_dict)